# CAB420, Week 2 Practical - Template

**Note, as the three questions in this weeks practical build on one another, all solutions are in the one script**

## Problem 1. Overfitting Linear Regression
In the week 1 practical, you developed a model to predict cyclist counts at a single counter. Using the same data and starting from the initial model before terms were removed, overcomplicate it such that it overfits to the training data. The easiest way to do this is by including a large number of higher order (i.e. interaction, quadratic and higher order polynomial) terms.

Verify that the model has overfit through evaluating on the validation and testing datasets, and compare it's performance to the simple model that you started with.

## Problem 2. Ridge Regression
Apply ridge regression to your two models (the simple model from Week 1, and the overfitting model from Problem 1 of this week). Using the validation set, select the best value of $\lambda$ for each model. For the selected model:
* Compute the $R^2$ and adjusted $R^2$, and draw a qqplot to assess the models validity;
* Compute the RMSE on the test set and compare the performance with the linear models.

## Problem 3. Lasso Regression
Apply lasso regression to your two models (the simple model from Week 1, and the overfitting model from Problem 1 of this week). Using the validation set, select the best value of $\lambda$ for each model. For the selected model:
* Compute the $R^2$ and adjusted $R^2$ (make sure to consider how many terms are removed by lasso), and draw a qqplot to assess the models validity;
* Compute the RMSE on the test set and compare the performance with the linear models and the ridge regression models.

### Relevant Examples

The second linear regression example, ``CAB420_Regression_Example_2_Regularised_Regression.ipynb`` is a useful starting point here. You may also find the third linear regression example, ``CAB420_Regression_Example_3_Regression_with_Less_Data.ipynb`` of use, however this contains the same relvant code (fitting linear and regularised regression).

### Suggested Packages

The following packages are suggested, however there are many ways to approach things in python, if you'd rather use different pacakges that's cool too.

In particular with this pratical, you have a choice of whether you'd rather use sklearn or statsmodels for regression.

In [1]:
# import all the important packages

# numpy handles pretty much anything that is a number/vector/matrix/array
import numpy as np
# pandas handles dataframes (exactly the same as tables in Matlab)
import pandas as pd
# matplotlib emulates Matlabs plotting functionality
import matplotlib.pyplot as plt
# seaborn, because of excellent heatmaps
import seaborn as sns;
# stats models is a package that is going to perform the regression analysis
from statsmodels import api as sm
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score
# can also use sklearn for our regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
# os allows us to manipulate variables on out local machine, such as paths and environment variables
import os
# self explainatory, dates and times
from datetime import datetime, date
# a helper package to help us iterate over objects
import itertools

/opt/conda/envs/cab420-env/lib/python3.9/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


### Step 1: Load and Pre-process the data

Loading and pre-processing should follow the same process as the previous practical. This should:
* load the data
* split into train, validation and test and pull out X and Y arrays

In this practical we also seek to overcomplicate our model, i.e. add higher order terms, etc, such that it overfits. To achieve this you could:
* Use ``PolynomialFeatures`` which has been imported from ``sklearn.preprocessing``; or
* Use the x2fx function below which replicates a similar function is MATLAB and provides a handy way of adding higher order terms.

Maintain the original (unexpanded) copy of the data as a separate variable as well as the "blown up" version.

You should also consider the issue of data standardisation. This will be useful for the regularised regression and should be done prior to Questions 2 and 3; but you may also wish to do this here such that you are using the same data throughout. A function for standardisation is provided below.

In [2]:
def x2fx(x, model='quadratic'):
  """Will create a quadratic, interaction and linear
  versions of our variable of interest

  Similar to the `x2fx()` function in Matlab.

  Written by Saullo G. P. Castro, from
  https://stackoverflow.com/questions/26574998/is-there-an-equivalent-function-for-x2fx-in-numpy

  Basically don't worry too much about how this func works. If I had to write it myself,
  it would look a lot worse than this.

  Args:
    x (np.array):
      Data set with columns as features (variables) that we 
      want to generate higher order terms for
    model (str):
      Determine linear, interaction, quadratic, purequadratic terms
 
  Returns:
    Array with higher order terms added as additional columns
  """
  from itertools import combinations as comb
  linear = np.c_[np.ones(x.shape[0]), x]
  if model == 'linear':
    return linear
  if model == 'purequadratic':
    return np.c_[linear, x**2]

  interaction = np.vstack([x[:,i]*x[:,j] for i, j in
                           comb(range(x.shape[1]), 2)]).T
    
  if model == 'interaction':
    return np.c_[linear, interaction]
  if model == 'quadratic':
    return np.c_[linear, interaction, x**2]

def standardise(data):
  """ Standardise/Normalise data to have zero mean and unit variance

  Args:
    data (np.array):
      data we want to standardise (usually covariates)

    Returns:
      Standardised data, mean of data, standard deviation of data
  """
  mu = np.mean(data, axis=0)
  sigma = np.std(data, axis=0)
  scaled = (data - mu) / sigma
  return scaled, mu, sigma

In [11]:
# load the data and split into train, validation, and test set
def load_data():
    data =  pd.read_csv("./combined.csv")
    print(data.head())
    M_train = int(data.shape[0] * 0.8)
    M_validation = int(data.shape[0] * 0.1)
    M_test = int(data.shape[0] * 0.1)
    X = []
    Y = []
    X.append(data.iloc[:M_train])
    X.append(data.iloc[:M_train])


    return X, Y

X,Y = load_data()
X[0].shape

   Unnamed: 0  Rainfall amount (millimetres)        Date  \
0           0                            0.0  2014-01-01   
1           1                            0.0  2014-01-02   
2           2                            1.0  2014-01-03   
3           3                            0.0  2014-01-04   
4           4                            0.0  2014-01-05   

   Maximum temperature (Degree C)  Daily global solar exposure (MJ/m*m)  \
0                            30.6                                  31.2   
1                            31.8                                  23.4   
2                            34.5                                  29.6   
3                            38.7                                  30.5   
4                            33.6                                  15.7   

   North Brisbane Bikeway Mann Park Windsor Cyclists Outbound  \
0                                                NaN            
1                                                NaN      

(1460, 57)

### Step 2 (Question 1): Fit the Linear Model

Fit a linear model to your data. Fit a model to both the original (i.e. without higher order terms) and expanded versions of the data to observe how severe the overfitting is.

Be sure to evaluate the models on the validation and test sets. You should also consider qq-plots and $R^2$ values when inspecting the model's performance. To maximise code re-use, you may wish to write a function that takes a model, and various datasets (train, validation and test) as input, and calculates relevant performance measures. Some functions to compute performance measures are given below. Remember that $R^2$ should only be computed on the training set.

In [3]:
def rmse(actual, pred):
  return np.sqrt(mean_squared_error(actual, pred))

def r_squared(actual, predicted):
  r2 = r2_score(actual, predicted)
  return r2

def adj_r2(actual, predicted, n, p):
  r2 = r2_score(actual, predicted)
  adjr2 = 1 - (1 - r2) * (n - 1) / (n - p - 1);
  return adjr2


### Step 3 (Question 2): Fit the Ridge Model

Fit a ridge model to your data.

It's recommended to use standardised data here, if you aren't already doing so.

The key consideration here is your value of $\lambda$. A good process to follow will be:
* Select a range of $\lambda$ values to consider. I would suggest using something like ``alpha_list = np.linspace(0, 10.0, 1000)``. Note that ``lambda`` is a keyword in python, hence we are calling this ``alpha`` here.
* Loop over the values of $\lambda$, i.e. ``for alpha in alpha_list:``
  * For each value of $\lambda$, fit a regression model
  * Get the RMSE on the validation set. You may also wish to compute:
    * RMSE on the training set
    * $R^2$ on the training set
* Find the value of $\lambda$ that minimises the RMSE on the validation set

If your best value of $\lambda$ is $0$, you need to select a smaller step in your list of values, (i.e. in the above example we'd change this to ``alpha_list = np.linspace(0, 0.1, 10.0)``). If your best value is equal the maximum value in your range, then you need to search over a larger area (i.e. in the above example change it to ``alpha_list = np.linspace(0, 50.0, 10000)``). You may also want to refine your estimate, without making the search space huge. To do this, let's say the you get a value of $\lambda$ of 100, using the above ``linspace``. In this case you could search around the following: ``alpha_list = np.linspace(90.0, 0.1, 110.0)``.

Once you've found your value of $\lambda$, fit the final model, evaluate it as you did for the linear model, and compare the results.

### Step 4 (Question 3): Fit the LASSO Model

Fit a LASSO model to your data.

Your approach here should really follow what you've done for ridge. You should be able to copy and paste except:
* Change your range of $\lambda$ values. The same values that worked for Ridge will not be optimal here
* Change the regression function that you are calling. If you are using sklearn, this means swapping ``sklearn.linear_model.Ridge`` for ``sklearn.linear_model.Lasso``. If you are using statsmodels, this means that when you call ``fit_regularized`` you pass in ``L1_wt=1``.

Notes regarding $\lambda$ values as the same as for Ridge regression.

Once you have your final value of $\lambda$, evaluate the model and compare to the others.